# Market Prices

Data obtained from [REN Datahub](https://mercado.ren.pt/PT/Electr/InfoMercado/InfOp/MercOmel/Paginas/Precos.aspx).

In [2]:
import warnings

import pandas as pd

warnings.filterwarnings('ignore')


In [6]:
df = pd.read_excel("../data/PrecoMerc.xlsx")
df.head()

,Date,Hour,Session,Price - PT [€/MWh],Price - ES [€/MWh]
0,2008-07-01,1,0,70.31,63.11
1,2008-07-01,2,0,64.54,58.55
2,2008-07-01,3,0,64.08,55.85
3,2008-07-01,4,0,63.50,55.00
4,2008-07-01,5,0,63.28,53.96


In [9]:
df.columns = ["date", "hour", "session", "price_pt", "price_es"]
df = df[df['session'] == 0][["date", "hour", "price_pt"]]
df.head()

,date,hour,price_pt
0,2008-07-01,1,70.31
1,2008-07-01,2,64.54
2,2008-07-01,3,64.08
3,2008-07-01,4,63.50
4,2008-07-01,5,63.28


In [10]:
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
df["hour"] = pd.to_timedelta(df["hour"], unit="h")
df["date"] = df["date"] + df["hour"]
df.head()

,date,hour,price_pt
0,2008-07-01 01:00:00,0 days 01:00:00,70.31
1,2008-07-01 02:00:00,0 days 02:00:00,64.54
2,2008-07-01 03:00:00,0 days 03:00:00,64.08
3,2008-07-01 04:00:00,0 days 04:00:00,63.50
4,2008-07-01 05:00:00,0 days 05:00:00,63.28


In [11]:
df = df[["date", "price_pt"]]
df.rename(columns={"price_pt": "price"}, inplace=True)
df.head()

,date,price
0,2008-07-01 01:00:00,70.31
1,2008-07-01 02:00:00,64.54
2,2008-07-01 03:00:00,64.08
3,2008-07-01 04:00:00,63.50
4,2008-07-01 05:00:00,63.28


In [12]:
df.to_csv("../data/MarketPricePT.csv", index=False)